In [ ]:
import os
import sys
import pandas as pd
import swifter

TIMESTAMPS = {
    'JAN': [1672531200, 1675209599],
    'FEB': [1675209600, 1677628799],
    'MAR': [1677628800, 1680307199],
    'APR': [1680307200, 1682899199],
    'MAY': [1682899200, 1685577599],
    'JUN': [1685577600, 1688169599],
    'JUL': [1688169600, 1690847999],
    'AUG': [1690848000, 1693526399],
    'SEP': [1693526400, 1696118399],
    'OCT': [1696118400, 1698796799],
    'NOV': [1698796800, 1701388799],
    'DEC': [1701388800, 1704067199],
}

input = pd.read_json('../data/input.json')


# Config

### Helper Methods

In [1]:
import os
import sys
import numpy as np
import swifter

MONTHS = ['MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP']

TIMESTAMPS = {
    'JAN': [1672531200, 1675209599],
    'FEB': [1675209600, 1677628799],
    'MAR': [1677628800, 1680307199],
    'APR': [1680307200, 1682899199],
    'MAY': [1682899200, 1685577599],
    'JUN': [1685577600, 1688169599],
    'JUL': [1688169600, 1690847999],
    'AUG': [1690848000, 1693526399],
    'SEP': [1693526400, 1696118399],
    'OCT': [1696118400, 1698796799],
    'NOV': [1698796800, 1701388799],
    'DEC': [1701388800, 1704067199],
}

## Input

In [2]:
import pandas as pd

df = pd.DataFrame()

for month in MONTHS:
  # continue if file does not exist
  if not os.path.exists(FP_FORMAT.format(month=month)):
    continue

  month_json = FP_FORMAT.format(month=month)
  df_month = pd.read_json(month_json)
  df_month['month'] = month
  df = pd.concat([df, df_month])


# drow rows where columns totalTransactions, uniqueNetworks, and totalVolume are all 0
print('dropping rows with empty metrics...\t=>', df.shape[0], 'wallets')
df = df[~((df['totalTransactions'] == 0) & (df['totalUniqueNetworks'] == 0) & (df['totalVolume'] == 0))].reset_index(drop=True)
print('remaining wallets...\t\t\t=>', df.shape[0], 'wallets')

print('\nDataFrame Preview:')
df.head()

dropping rows with empty metrics...	=> 1516861 wallets
remaining wallets...			=> 431610 wallets

DataFrame Preview:


,wallet,totalTransactions,totalUniqueNetworks,totalVolume,month
0,0x7d4588C83df708008E5c3F3eCe3682e266fF1Cc1,4,2,4903.85,MAR
1,0x78EFF7aF39b5B547eDfbA0fb07B55ed147Ee81Ad,6,2,991.98,MAR
2,0xddA42f12B8B2ccc6717c053A2b772baD24B08CbD,24,5,12610.30,MAR
3,0x988527874C7E3f02115F89A6a97135c70B6a47Fc,2,1,42.18,MAR
4,0xC76A6a6Ee1d8514F4AE9AA223874C254cBDAB8DC,1,1,23.30,MAR


In [3]:
df = df[df['month'] == 'AUG']
df.shape

(51953, 5)

# Aggregation

### PDAs By Month

In [5]:
df_pda = df.groupby(['wallet', 'month']).sum().reset_index()
df_pda.head()

,wallet,month,totalTransactions,totalUniqueNetworks,totalVolume
0,0x00000005bb0505fABBF8514f8c260103F41e6777,AUG,3,1,472.42
1,0x0000001089167600c25258Da29d2e2c857ec1689,AUG,2,2,2150.77
2,0x0000002396544aC00eE0c1A4427f5c1134E79883,AUG,1,1,735.83
3,0x0000006D14cE3CF81449c3BA1f26108DF0A4de8b,AUG,2,2,35.00
4,0x000000866f4E8ac304B389E46a6a4e2D41694D96,AUG,1,1,0.53


### Loyalty Passes

In [6]:
df_lp = df.groupby('wallet').sum().reset_index()[['wallet', 'totalTransactions', 'totalUniqueNetworks', 'totalVolume']]
df_lp

,wallet,totalTransactions,totalUniqueNetworks,totalVolume
0,0x00000005bb0505fABBF8514f8c260103F41e6777,3,1,472.42
1,0x0000001089167600c25258Da29d2e2c857ec1689,2,2,2150.77
2,0x0000002396544aC00eE0c1A4427f5c1134E79883,1,1,735.83
3,0x0000006D14cE3CF81449c3BA1f26108DF0A4de8b,2,2,35.00
4,0x000000866f4E8ac304B389E46a6a4e2D41694D96,1,1,0.53
...,...,...,...,...
51948,0xffa35D5fB9EfcF5B546DcDEac5dC6fF52e6E40B9,1,1,1.91
51949,0xffaaA741C66D276921990F9360538D0B4e73D856,1,1,8.88
51950,0xffc3f8b3F811aecE90af61FbF5Ac91F22327e8fd,3,2,9.80
51951,0xffeeE60c31FdB5036E0D5bf4DE86854BbCcd9f7A,4,3,39.60


# Export

In [7]:
import time

now = int(time.time())
export_fp = f'./exports/{now}-pda.json'
df_pda.to_json(export_fp, orient='records')